In [ ]:
# Homework 6 (due 08/08/2024)

# Neural networks and computer vision

### Objective
In this week's project, you will learn to train, validate, and test a neural network. You will explore how inputs change through feature extraction in convolutional neural networks (CNNs), and you will interpret the trained filters by the network.

#### Dataset

You will use the MNIST dataset, a standard dataset of handwritten digits, which is widely used for training and testing image processing systems.

#### Instructions

The code example below demonstrates how to define, train, validate, and test a CNN. The training and test accuracy after each completed epoch are shown after a completed

**1. Explore a working example**
1. Open `example.ipynb` and read the code.
2. Consult the pytorch documentation to learn what the arguments of the various employed pytorch functions mean.
3. Run the code.
4. Replace SGD with Adam in the training process. Then run the code again.
5. Save the output figures that show training and validation accuracy as a function of the number of epochs in your file system.

**2. Build a network**
Create your own working example. (You are allowed to copy any amount of code from `example.ipynb`.) Your CNN should be different from the CNN in the working example in the following ways:
1. The new CNN should have three convolutional layers instead of two. The first layer creates 32 channels. The second layer creates 64 channels, and the third layer creates 128 channels.
2. The pooling layer after the third layer should not employ any padding.
3. The last hidden layer should have 512 neurons.
4. For all layers except the output layer, the activation function should be a ReLU (use `torch.nn.ReLU`).

**3. Train and evaluate a neural network**
1. Train the neural network that you have constructed in the previous step. How have the upgrades with respect to the CNN in `example.ipynb` affected the CNN's training time?
2. Test the neural network. How have the upgrades with respect to the CNN in `example.ipynb` affected the CNN's validation accuracy?
3. Identify the number $k$ of training epochs that gives you a good tradeoff between training time and validation accuracy.
4. Run your code again using $k$ epochs during training. Time the training (e.g. using the python library `time`).

**4. Model validation and model selection**
1. Use the validation set approach to identify the best number $c$ of channels in the first convolutional layer (consider $c\in\{2,15\}$).
2. Update your neural network architecture so that the first convolutional layer has $c$ channels.

**5. Visualizing feature extraction**
1. Use the function `plot_mapped_features` to view an input image and the corresponding first channel of the hidden state for each feature-extraction layer (i.e., each convolution layer and each pooling layer).
2. Update the function so that it shows all channels instead of just one.
3. Comment on where you observe differences between the channels within a layer.

**6. Visualizing and interpreting filters**
1. Use the function `plot_filters` to view the trained filters of the first convolutional layer.
2. Identify the filters that perform blurring, sharpening, or horizontal or vertical edge detection.

**7. Comparison to logistic regression**
1. Construct and run a pipeline for multiclass logistic regression of the MNIST dataset using sklearn.
2. Comment on how the training time and test accuracy of logistic regression compare to the CNN.
3. Now run multiclass logistic regression on the MNIST data set using one of the hidden states of the CNN (i.e., $\vec{x}^{(2)}$, $\vec{x}^{(3)}$, ..., $\vec{x}^{(7)}$) as inputs. Which set of inputs yields the best classification results?

In [21]:
# import packages
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt
import numpy as np

In [22]:
# Load and preprocess the MNIST dataset

# Define an array transformation that transforms the images to tensor format 
# and normalizes the pixel values to the range [-1, 1]
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Download and load the training and test datasets
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, 
    download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, 
    download=True, transform=transform)

# Split the training dataset into a training set and a validation set
train_set, val_set = random_split(train_dataset, [50000, 10000])

# Create data loaders for the training, validation, and test sets
# A DataLoader in PyTorch is an object that simplifies and automates
# batching, shuffling, and loading data for model training and evaluation. 
train_loader = DataLoader(train_set, batch_size=128, shuffle=True)
val_loader = DataLoader(val_set, batch_size=128, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

In [34]:
"""
Part 2. Update CNN
"""
# Define CNN architecture

class CNN(nn.Module):
    def __init__(self, c):
        """
        Initialize the CNN model by defining its layers.
        """
        # Create an instance of the parent class `nn.Module`
        super(CNN, self).__init__()  
        self.c = c
        self.conv1 = nn.Conv2d(1, c, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(c, c*2, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(c*2, c*4, kernel_size=3, padding=1)
        # Define the activation function
        self.activation = nn.ReLU()
        # Define a pooling layer
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        # Inputs are num_channels in previous layer x image height x image width
        self.fc1 = nn.Linear(c*4 * 3 * 3, 512)
        # Define the output layer with 10 nodes
        self.fc2 = nn.Linear(512, 10)
        
    def forward(self, x):
        """
        Define the forward pass of the CNN.

        Parameters:
        x : torch.Tensor
            The input tensor containing the image batch.

        Returns:
        torch.Tensor
            The output tensor containing the class scores for each image.
        """
        # Pass the input through the first convolutional layer, then apply activation
        x = self.activation(self.conv1(x))
        # Pass the input through the first pooling layer
        x = self.pool(x)
        # Pass the input through the second convolutional layer, then apply activation
        x = self.activation(self.conv2(x))
        x = self.pool(x)
        # Pass the input through the second pooling layer
        x = self.activation(self.conv3(x))
        x = self.pool(x)     
        # Change the shape of x into a 1d array
        x = x.view(-1, self.c*4 * 3 * 3)
        # Pass the input through the full connected hidden layer, then apply activation
        x = self.activation(self.fc1(x))
        # Pass the input through the last layer
        x = self.fc2(x)
        return x

In [35]:
# Define training pipeline including validation after each epoch
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=10):
    """
    Train the CNN model.

    Parameters:
    model : torch.nn.Module
        The CNN model to be trained.
    train_loader : torch.utils.data.DataLoader
        The data loader for the training set.
    val_loader : torch.utils.data.DataLoader
        The data loader for the validation set.
    criterion : torch.nn.modules.loss._Loss
        The loss function to be used.
    optimizer : torch.optim.Optimizer
        The optimizer to be used.
    epochs : int
        The number of epochs for training.

    Returns:
    tuple
        A tuple containing lists of training loss, validation loss, training accuracy, and validation accuracy.
    """
    # Initialize lists to store training and validation loss
    train_loss, val_loss = [], []
    # Initialize lists to store training and validation and accuracy
    train_acc, val_acc = [], []

    # Loop over the number of epochs
    for epoch in range(epochs):
        # Set the model to training mode
        model.train()  
        # Initialize the running loss for the epoch
        running_loss = 0.0  
        # Initialize counters for correct predictions and total samples
        correct, total = 0, 0  

        # Learning algorithm is SGD with minibatch. Iterating over the dataload
        # returns images and labels in batches.
        
        # Iterate over batches of training data
        for images, labels in train_loader:
            # Zero the gradients to prevent accumulation from previous iterations
            optimizer.zero_grad()  
            # Perform a forward pass through the model to get predictions
            outputs = model(images)  
            # Compute the loss between predictions and true labels
            loss = criterion(outputs, labels)  
            # Perform a backward pass to compute gradients via backpropagation
            loss.backward()  
            # Update model parameters based on the computed gradients
            optimizer.step()  

            # Add up the loss
            running_loss += loss.item()  
            # Get the predicted class with the highest score
            _, predicted = torch.max(outputs.data, 1)  
            # Update the total number of samples
            total += labels.size(0)  
            # Update the number of correct predictions
            correct += (predicted == labels).sum().item()  

        # Compute and store the average training loss for the epoch
        train_loss.append(running_loss / len(train_loader))  
        # Compute and store the training accuracy for the epoch
        train_acc.append(100 * correct / total)  

        # Set the model to evaluation mode
        model.eval()  
        # Initialize the running loss for validation
        val_running_loss = 0.0  
        #  Initialize counters for correct predictions and total samples in validation
        val_correct, val_total = 0, 0  
        
        # Disable gradient calculation for validation to save memory and computation
        with torch.no_grad():
            # Iterate over batches of validation data
            for images, labels in val_loader:
                # Perform a forward pass through the model to get predictions
                outputs = model(images)  
                # Compute the loss between predictions and true labels
                loss = criterion(outputs, labels)  
                # Add up the loss
                val_running_loss += loss.item()  
                # Get the predicted class with the highest score
                _, predicted = torch.max(outputs.data, 1)  
                # Update the total number of samples in validation
                val_total += labels.size(0)  
                # Update the number of correct predictions in validation
                val_correct += (predicted == labels).sum().item()  

        # Compute and store the average validation loss for the epoch
        val_loss.append(val_running_loss / len(val_loader))
        # Compute and store the validation accuracy for the epoch
        val_acc.append(100 * val_correct / val_total)  
        
        # Print the results for the current epoch, including training and validation loss and accuracy
        print(f'Epoch [{epoch+1}/{epochs}], Train Loss: {running_loss / len(train_loader):.4f}, '
              f'Validation Loss: {val_running_loss / len(val_loader):.4f}, '
              f'Train Acc: {100 * correct / total:.2f}%, Val Acc: {100 * val_correct / val_total:.2f}%')
        
    # Return the lists of training and validation loss and accuracy
    return train_loss, val_loss, train_acc, val_acc  

In [41]:
# Build and train a model
from time import perf_counter

epochs = 10

c_list = [2,4,6,8,10,12,14]
for c in c_list:
    print("c =", c)
    start = perf_counter()
    # Create model
    model = CNN(c=c)

    # Set loss function
    criterion = nn.CrossEntropyLoss()

    # Set training algorithm
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Train model
    train_loss, val_loss, train_acc, val_acc = train_model(model, train_loader, val_loader, criterion, optimizer, epochs = epochs)

    end = perf_counter()
    print("time to run was", end - start, "s")


c = 2
Epoch [1/10], Train Loss: 0.7015, Validation Loss: 0.2484, Train Acc: 78.19%, Val Acc: 92.49%
Epoch [2/10], Train Loss: 0.1884, Validation Loss: 0.1677, Train Acc: 94.16%, Val Acc: 94.94%
Epoch [3/10], Train Loss: 0.1426, Validation Loss: 0.1321, Train Acc: 95.56%, Val Acc: 95.65%
Epoch [4/10], Train Loss: 0.1163, Validation Loss: 0.1143, Train Acc: 96.29%, Val Acc: 96.49%
Epoch [5/10], Train Loss: 0.0995, Validation Loss: 0.0926, Train Acc: 96.90%, Val Acc: 97.23%
Epoch [6/10], Train Loss: 0.0880, Validation Loss: 0.0909, Train Acc: 97.22%, Val Acc: 97.15%
Epoch [7/10], Train Loss: 0.0815, Validation Loss: 0.0820, Train Acc: 97.44%, Val Acc: 97.39%
Epoch [8/10], Train Loss: 0.0730, Validation Loss: 0.0857, Train Acc: 97.65%, Val Acc: 97.32%
Epoch [9/10], Train Loss: 0.0682, Validation Loss: 0.0794, Train Acc: 97.84%, Val Acc: 97.46%
Epoch [10/10], Train Loss: 0.0631, Validation Loss: 0.0766, Train Acc: 97.88%, Val Acc: 97.63%
time to run was 129.76415270904545 s
c = 4
Epoch [1/1

# Part 3

The upgrades to the CNN in example.ipynb have significantly increased the training time. Whereas with the Adam algorithm and the example CNN, the training time for all 10 iterations of training epochs took ~2.5 minutes, the updated CNN, also using Adam, ran for 10 minutes for the first five iterations of training epochs (1-5), after which I terminated the run.

The validation accuracy is much higher for the upgraded CNN. 
The example CNN yielded the following accuracy rates:

Epoch [1/10], Train Loss: 1.3627, Validation Loss: 0.4408, Train Acc: 56.16%, Val Acc: 87.99%
Epoch [2/10], Train Loss: 0.3149, Validation Loss: 0.2286, Train Acc: 91.31%, Val Acc: 93.59%
Epoch [3/10], Train Loss: 0.1970, Validation Loss: 0.1635, Train Acc: 94.37%, Val Acc: 95.32%
Epoch [4/10], Train Loss: 0.1496, Validation Loss: 0.1317, Train Acc: 95.61%, Val Acc: 96.22%
Epoch [5/10], Train Loss: 0.1213, Validation Loss: 0.1093, Train Acc: 96.42%, Val Acc: 96.94%
Epoch [6/10], Train Loss: 0.1020, Validation Loss: 0.0973, Train Acc: 97.00%, Val Acc: 97.15%
Epoch [7/10], Train Loss: 0.0890, Validation Loss: 0.0885, Train Acc: 97.31%, Val Acc: 97.40%
Epoch [8/10], Train Loss: 0.0792, Validation Loss: 0.0805, Train Acc: 97.62%, Val Acc: 97.61%
Epoch [9/10], Train Loss: 0.0708, Validation Loss: 0.0793, Train Acc: 97.94%, Val Acc: 97.60%
Epoch [10/10], Train Loss: 0.0631, Validation Loss: 0.0686, Train Acc: 98.15%, Val Acc: 97.94%

The upgraded CNN yielded the following accuracy rates for the first five epochs:

Epoch [1/10], Train Loss: 0.2155, Validation Loss: 0.0613, Train Acc: 93.20%, Val Acc: 98.28%
Epoch [2/10], Train Loss: 0.0524, Validation Loss: 0.0473, Train Acc: 98.37%, Val Acc: 98.53%
Epoch [3/10], Train Loss: 0.0338, Validation Loss: 0.0478, Train Acc: 98.95%, Val Acc: 98.61%
Epoch [4/10], Train Loss: 0.0252, Validation Loss: 0.0375, Train Acc: 99.18%, Val Acc: 98.94%
Epoch [5/10], Train Loss: 0.0219, Validation Loss: 0.0346, Train Acc: 99.30%, Val Acc: 98.98%

With just one epoch, the validation accuracy was higher (98.28%) than the validation accuracy after ten epochs with the original CNN (97.94%)

Additionally, the testing accuracy also increased with the upgraded CNN. Whil the testing accuracy for the example CNN was 98.16%, the testing accuracy was 98.91% for the upgraded CNN; however, this increase was less impressive.

I identified k = 2 as a good mark for tradeoff between training time and validation accuracy. Validation was around 98.5% while the training time was less than five minutes.
This was my output:
Epoch [1/2], Train Loss: 0.2160, Validation Loss: 0.0569, Train Acc: 93.21%, Val Acc: 98.26%
Epoch [2/2], Train Loss: 0.0480, Validation Loss: 0.0514, Train Acc: 98.49%, Val Acc: 98.46%
time to run was 292.9701649999479 s

Part 4 \
c = 2 \
Epoch [1/5], Train Loss: 0.6116, Validation Loss: 0.2242, Train Acc: 81.47%, Val Acc: 93.15% \
Epoch [2/5], Train Loss: 0.1760, Validation Loss: 0.1475, Train Acc: 94.60%, Val Acc: 95.36% \
Epoch [3/5], Train Loss: 0.1337, Validation Loss: 0.1341, Train Acc: 95.76%, Val Acc: 95.71%
Epoch [4/5], Train Loss: 0.1091, Validation Loss: 0.0933, Train Acc: 96.46%, Val Acc: 97.05%
Epoch [5/5], Train Loss: 0.0953, Validation Loss: 0.0889, Train Acc: 96.98%, Val Acc: 97.28%
time to run was 63.234789874986745 s
c = 4
Epoch [1/5], Train Loss: 0.4578, Validation Loss: 0.1374, Train Acc: 86.65%, Val Acc: 95.72%
Epoch [2/5], Train Loss: 0.1144, Validation Loss: 0.0888, Train Acc: 96.40%, Val Acc: 97.30%
Epoch [3/5], Train Loss: 0.0845, Validation Loss: 0.0736, Train Acc: 97.40%, Val Acc: 97.83%
Epoch [4/5], Train Loss: 0.0662, Validation Loss: 0.0737, Train Acc: 97.87%, Val Acc: 97.72%
Epoch [5/5], Train Loss: 0.0577, Validation Loss: 0.0651, Train Acc: 98.15%, Val Acc: 98.01%
time to run was 75.20662604202516 s
c = 6
Epoch [1/5], Train Loss: 0.4566, Validation Loss: 0.1385, Train Acc: 85.52%, Val Acc: 95.78%
Epoch [2/5], Train Loss: 0.1029, Validation Loss: 0.0883, Train Acc: 96.74%, Val Acc: 97.30%
Epoch [3/5], Train Loss: 0.0714, Validation Loss: 0.0832, Train Acc: 97.77%, Val Acc: 97.31%
Epoch [4/5], Train Loss: 0.0575, Validation Loss: 0.0578, Train Acc: 98.21%, Val Acc: 98.30%
Epoch [5/5], Train Loss: 0.0454, Validation Loss: 0.0548, Train Acc: 98.58%, Val Acc: 98.40%
time to run was 89.28104862500913 s
c = 8
Epoch [1/5], Train Loss: 0.3842, Validation Loss: 0.1071, Train Acc: 88.15%, Val Acc: 96.75%
Epoch [2/5], Train Loss: 0.0875, Validation Loss: 0.0738, Train Acc: 97.27%, Val Acc: 97.75%
Epoch [3/5], Train Loss: 0.0634, Validation Loss: 0.0554, Train Acc: 97.99%, Val Acc: 98.15%
Epoch [4/5], Train Loss: 0.0486, Validation Loss: 0.0709, Train Acc: 98.45%, Val Acc: 97.88%
Epoch [5/5], Train Loss: 0.0410, Validation Loss: 0.0428, Train Acc: 98.65%, Val Acc: 98.80%
time to run was 104.57447908399627 s
c = 10
Epoch [1/5], Train Loss: 0.3868, Validation Loss: 0.0922, Train Acc: 87.96%, Val Acc: 97.28%
Epoch [2/5], Train Loss: 0.0802, Validation Loss: 0.0639, Train Acc: 97.58%, Val Acc: 98.11%
Epoch [3/5], Train Loss: 0.0540, Validation Loss: 0.0539, Train Acc: 98.28%, Val Acc: 98.37%
Epoch [4/5], Train Loss: 0.0437, Validation Loss: 0.0438, Train Acc: 98.62%, Val Acc: 98.65%
Epoch [5/5], Train Loss: 0.0354, Validation Loss: 0.0581, Train Acc: 98.89%, Val Acc: 98.23%
time to run was 118.20634833304211 s
c = 12
Epoch [1/5], Train Loss: 0.3251, Validation Loss: 0.0867, Train Acc: 90.08%, Val Acc: 97.31%
Epoch [2/5], Train Loss: 0.0677, Validation Loss: 0.0487, Train Acc: 97.89%, Val Acc: 98.55%
Epoch [3/5], Train Loss: 0.0469, Validation Loss: 0.0512, Train Acc: 98.52%, Val Acc: 98.34%
Epoch [4/5], Train Loss: 0.0366, Validation Loss: 0.0411, Train Acc: 98.82%, Val Acc: 98.66%
Epoch [5/5], Train Loss: 0.0279, Validation Loss: 0.0359, Train Acc: 99.08%, Val Acc: 98.90%
time to run was 132.7757134580752 s
c = 14
Epoch [1/5], Train Loss: 0.2967, Validation Loss: 0.0792, Train Acc: 90.90%, Val Acc: 97.44%
Epoch [2/5], Train Loss: 0.0662, Validation Loss: 0.0587, Train Acc: 97.88%, Val Acc: 98.18%
Epoch [3/5], Train Loss: 0.0449, Validation Loss: 0.0413, Train Acc: 98.55%, Val Acc: 98.81%
Epoch [4/5], Train Loss: 0.0373, Validation Loss: 0.0382, Train Acc: 98.81%, Val Acc: 98.91%
Epoch [5/5], Train Loss: 0.0269, Validation Loss: 0.0354, Train Acc: 99.15%, Val Acc: 98.95%
time to run was 198.1088390829973 s



In [4]:
# Function to visualize the feature maps produced by different layers for a given image
def plot_mapped_features(model, image, layers):
    '''Example usage: 
    
    >>> examples = iter(test_loader)
    >>> example_data, example_labels = next(examples) # get one batch from test set
    >>> example_image = example_data[0]
    >>> layers = [model.conv1, model.pool, model.conv2, model.pool]
    >>> plot_mapped_features(model, example_image, layers)
    
    '''
    # Add a batch dimension to the image tensor (from (channels, height, width) to (1, channels, height, width))
    x = image.unsqueeze(0)
    # Create a subplot with 1 row and len(layers) columns
    fig, axes = plt.subplots(1, len(layers))
    # Iterate over the specified layers
    for i, layer in enumerate(layers):
        # Pass the image through the current layer
        x = layer(x)
        # Detach the feature map from the computation graph and move it to CPU, then convert it to a NumPy array
        # Visualize the first channel of the feature map
        axes[i].imshow(x[0, 0].detach().cpu().numpy(), cmap='gray')
        # Turn off the axis for a cleaner look
        axes[i].axis('off')
    # Display the feature maps
    plt.show()
    
# Function to visualize the filters of a given convolutional layer
def plot_filters(layer, n_filters=6):
    '''Example usage: 

    >>> layer = model.conv1
    >>> plot_filters(layer, n_filters=6)
    
    '''
    # Clone the weights of the convolutional layer to avoid modifying the original weights
    filters = layer.weight.data.clone()
    # Normalize the filter values to the range [0, 1] for better visualization
    filters = filters - filters.min()
    filters = filters / filters.max()
    # Select the first n_filters to visualize
    filters = filters[:n_filters]
    # Create a subplot with 1 row and n_filters columns
    fig, axes = plt.subplots(1, n_filters)
    # Iterate over the selected filters
    for i, filter in enumerate(filters):
        # Transpose the filter dimensions to (height, width, channels) for visualization
        axes[i].imshow(np.transpose(filter, (1, 2, 0)))
        # Turn off the axis for a cleaner look
        axes[i].axis('off')
    # Display the filters
    plt.show()

In [26]:
# Evaluate the model on test set

model.eval()
test_correct, test_total = 0, 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()

test_acc = 100 * test_correct / test_total
print(f'Test Accuracy: {test_acc:.2f}%')

Test Accuracy: 98.91%
